 # BabyLLaMA

Coding the LLaMA-2 research paper from scratch to create models with sizes 100M, 250M and 500M params.

## Model Arch

Decoder only: Composed of identical `n_layers`. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple position-wise fully connected FFN. We employ residual connection around each of the sub-layers, followed by layers normalizatin. That is:
LayerNorm(x + Sublayer(x))
 -- A Vaswani et al., 2017.

In [1]:
!pip install -U -q accelerate transformers[torch] datasets huggingface_hub torchviz torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
def flush():
    gc.collect()
    torch.cuda.empty_cache()

def count_parameters(model):
    return f"BabyLlama size: {sum(p.numel() for p in model.parameters() if p.requires_grad) / 10 ** 6:.2f}M parameters"

def visualize_model(model, outputs):
    make_dot(outputs, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")


In [3]:
import gc
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from torchviz import make_dot
from math import sqrt
from transformers import PretrainedConfig
import math
from typing import Tuple, Optional, List
from transformers import logging, PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("CUDA is available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
logger = logging.get_logger(__name__)


CUDA is available: True
Current device: 0
Device name: NVIDIA GeForce GTX 1650


In [4]:
class BabyLlamaConfig(PretrainedConfig):
    model_type = "llama"
    keys_to_ignore_at_inference = ["past_key_values"]
    def __init__(
        self,
        vocab_size=32000,
        hidden_size=1024, # 2048 Tiny LLaMA
        intermediate_size=2048,
        num_hidden_layers=6,
        num_attention_heads=16, # 32 Tiny LLaMA
        num_key_value_heads=2,
        hidden_act="silu",
        max_position_embeddings=512,
        initializer_range=0.02,
        rms_norm_eps=1e-6,
        use_cache=False,
        pad_token_id=None,
        bos_token_id=1,
        eos_token_id=2,
        pretraining_tp=1,
        tie_word_embeddings=False,
        rope_theta=10000.0,
        rope_scaling=None,
        attention_bias=False,
        attention_dropout=0.0,
        use_bias=False,
        lm_head_bias=False,
        residual_dropout=0.0,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        **kwargs,
    ):

        self.vocab_size = vocab_size
        self.max_position_embeddings = max_position_embeddings
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads

        self.num_key_value_heads = num_key_value_heads
        self.hidden_act = hidden_act
        self.initializer_range = initializer_range
        self.rms_norm_eps = rms_norm_eps
        self.pretraining_tp = pretraining_tp
        self.use_cache = use_cache
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.attention_bias = attention_bias
        self.attention_dropout = attention_dropout
        self.residual_dropout = residual_dropout
        self.use_bias = use_bias
        self.lm_head_bias = lm_head_bias
        self.device = device

        super().__init__(
            bos_token_id=bos_token_id,
            eos_token_id=eos_token_id,
            **kwargs,
        )

### MHA
<img src="https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png" width=500>

- MQA
- GQA

In [5]:
def build_mask_cache(max_seq_length: int, device: Optional[torch.device] = None) -> torch.Tensor:
    ones = torch.ones((max_seq_length, max_seq_length), device=device, dtype=torch.bool)
    return torch.tril(ones).unsqueeze(0).unsqueeze(0)

def repeat_kv(hidden_states:torch.tensor, n_repeats:int):
    batch, n_kv_heads, seq_len, head_dim = hidden_states.shape
    if n_repeats == 1:
        return hidden_states
    hidden_states = hidden_states.unsqueeze(2).expand(batch, n_kv_heads, n_repeats, seq_len, head_dim) # (B, nh, T, hs) -> (B, nh, 1, T, hs) -> # (B, nh, n_repeats, T, hs)
    return hidden_states.reshape(batch, n_kv_heads * n_repeats, seq_len, head_dim) # # (B, nh * n_repeats, T, hs)


class RotaryPositionalEmbeddings(nn.Module):
    def __init__(self, dim, max_position_embeddings=512, base=10000, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'), scaling_factor=1.0):
        super().__init__()
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.device=device
        self.scaling_factor = scaling_factor
        self.base = base
        inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2, dtype=torch.int64).float().to(device) / self.dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)

        # Build here to make `torch.jit.trace` work.
        self._set_cos_sin_cache(
            seq_len=max_position_embeddings, device=self.inv_freq.device, dtype=torch.get_default_dtype()
        )

    def _set_cos_sin_cache(self, seq_len, device, dtype):
        self.max_seq_len_cached = seq_len
        t = torch.arange(self.max_seq_len_cached, device=device, dtype=torch.int64).type_as(self.inv_freq)
        t = t / self.scaling_factor
        freqs = torch.outer(t, self.inv_freq)
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos().to(dtype), persistent=False)
        self.register_buffer("sin_cached", emb.sin().to(dtype), persistent=False)

    @torch.no_grad()
    def forward(self, x, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        if seq_len > self.max_seq_len_cached:
            self._set_cos_sin_cache(seq_len=seq_len, device=x.device, dtype=x.dtype)

        return (
            self.cos_cached[:seq_len].to(dtype=x.dtype),
            self.sin_cached[:seq_len].to(dtype=x.dtype),
        )

    def apply_rope(self, x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor, position_ids, unsqueeze_dim=1) -> torch.Tensor:
        cos = cos[position_ids].unsqueeze(unsqueeze_dim)
        sin = sin[position_ids].unsqueeze(unsqueeze_dim)
        x1 = x[..., : x.shape[-1] // 2] # (B, nh, T, hs/2)
        x2 = x[..., x.shape[-1] // 2 :] # (B, nh, T, hs/2)
        rotated = torch.cat((-x2, x1), dim=-1) # (B, nh, T, hs)
        roped = (x * cos) + (rotated * sin)
        return roped.to(dtype=x.dtype)


    @property
    def sin_cached(self):
        logger.warning_once(
            "The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use "
            "the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class"
        )
        return self._sin_cached

    @property
    def cos_cached(self):
        logger.warning_once(
            "The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use "
            "the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class"
        )
        return self._cos_cached


class KVCache(nn.Module):
    def __init__(
        self,
        k_shape: Tuple[int, int, int, int],
        v_shape: Tuple[int, int, int, int],
        device: Optional[torch.device] = None,
        dtype: Optional[torch.dtype] = None,
    ) -> None:
        super().__init__()
        self.register_buffer("k", torch.zeros(k_shape, device=device, dtype=dtype), persistent=False)
        self.register_buffer("v", torch.zeros(v_shape, device=device, dtype=dtype), persistent=False)

    def forward(self, input_pos: torch.Tensor, k: torch.Tensor, v: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # move the buffer to the activation dtype for when AMP is used
        self.k = self.k.to(k.dtype)
        self.v = self.v.to(v.dtype)
        # update the cache
        k = self.k.index_copy_(2, input_pos, k)
        v = self.v.index_copy_(2, input_pos, v)
        return k, v

    def reset_parameters(self) -> None:
        torch.nn.init.zeros_(self.k)
        torch.nn.init.zeros_(self.v)

In [6]:
# KV caching
batch_size = 2
seq_len = 2
head_dim = 3
k_cache = torch.zeros([batch_size, seq_len, head_dim], device='cuda')
k = torch.rand([batch_size, seq_len, head_dim], device='cuda')
position_ids = torch.arange(k.size(2), dtype=torch.long, device='cuda')

k_cache.index_copy_(2, position_ids, k)

tensor([[[0.5398, 0.1765, 0.5165],
         [0.5673, 0.6294, 0.3317]],

        [[0.7566, 0.8971, 0.2973],
         [0.4891, 0.2296, 0.7631]]], device='cuda:0')

In [7]:
class BabyLlamaAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.hidden_dim = hidden_dim = config.hidden_size
        self.n_heads = n_heads = config.num_attention_heads
        self.n_kv_heads = n_kv_heads = config.num_key_value_heads
        self.head_dim = head_dim = config.hidden_size // n_heads
        use_bias = config.use_bias

        if (head_dim * n_heads) != self.hidden_dim:
            raise ValueError(
                f"hidden_dim must be divisible by num_heads (got `hidden_dim`: {self.hidden_dim}"
                f" and `num_heads`: {self.n_heads})."
            )

        self.repeats = n_heads // n_kv_heads # q_per_kv

        self.q_proj = nn.Linear(hidden_dim, n_heads * head_dim, bias=use_bias)
        self.k_proj = nn.Linear(hidden_dim, n_kv_heads * head_dim, bias=use_bias)
        self.v_proj = nn.Linear(hidden_dim, n_kv_heads * head_dim, bias=use_bias)
        self.o_proj = nn.Linear(n_heads * head_dim, hidden_dim, bias=use_bias)

        self.rotary_emb = RotaryPositionalEmbeddings(
            head_dim,
            max_position_embeddings=config.max_position_embeddings,
            device=config.device,
            base=config.rope_theta,
        )

        self.kv_cache: Optional[KVCache] = None

    def forward(
        self,
        hidden_states: torch.Tensor,
        mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
    ):
        B, T, _ = hidden_states.size() # bsz, seq_len, embed_dim

        queries = self.q_proj(hidden_states)
        keys = self.k_proj(hidden_states)
        values = self.v_proj(hidden_states)


        queries = queries.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)  # bsz, seq_len, n_heads, head_dim
        keys = keys.view(B, T, self.n_kv_heads, self.head_dim).transpose(1, 2) # bsz, seq_len, n_kv_heads, head_dim
        values = values.view(B, T, self.n_kv_heads, self.head_dim).transpose(1, 2)

        kv_seq_len = keys.shape[-2]
        cos, sin = self.rotary_emb(values, seq_len=kv_seq_len)

        queries = self.rotary_emb.apply_rope(queries, cos, sin, position_ids)
        keys = self.rotary_emb.apply_rope(keys, cos, sin, position_ids)


        # TODO: KV caching
        keys = repeat_kv(keys, self.repeats)
        values = repeat_kv(values, self.repeats)

        # SDPA with memory-efficient backend is currently (torch==2.1.2) bugged with non-contiguous inputs with custom attn_mask,
        # Reference: https://github.com/pytorch/pytorch/issues/112577.
        if queries.device.type == "cuda" and mask is not None:
            queries = queries.contiguous()
            keys = keys.contiguous()
            values = values.contiguous()

        y = self.scaled_dot_product_attention(queries, keys, values, mask) # (B, T, n_heads, head_dim)

        y = y.reshape(B, T, self.hidden_dim) # (B, T, hidden_dim)

        return self.o_proj(y)


    def scaled_dot_product_attention(
        self, q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:

        print()
        scale = 1.0 / math.sqrt(self.head_dim)
        y = torch.nn.functional.scaled_dot_product_attention(
            q, k, v, attn_mask=None, dropout_p=0.0, scale=scale, is_causal=True
        )
        return y.transpose(1, 2).contiguous()


In [8]:
config = BabyLlamaConfig(device='cuda' if torch.cuda.is_available() else 'cpu')

# Generate random input data
d_model=config.hidden_size
sequence_length = config.max_position_embeddings # number of tokens
batch_size = 5
input_data = torch.rand((batch_size, sequence_length, d_model), device=config.device)
position_ids = torch.arange(sequence_length, dtype=torch.long, device=config.device).unsqueeze(0)

attn = BabyLlamaAttention(config).to(config.device)
attn(input_data,position_ids=position_ids).shape

The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


torch.Size([5, 512, 1024])

In [9]:
attn

BabyLlamaAttention(
  (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
  (k_proj): Linear(in_features=1024, out_features=128, bias=False)
  (v_proj): Linear(in_features=1024, out_features=128, bias=False)
  (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
  (rotary_emb): RotaryPositionalEmbeddings()
)

In [10]:
count_parameters(attn)

'BabyLlama size: 2.36M parameters'

In [11]:
class LLaMAMLP(nn.Module):
    def __init__(self, hidden_dim, intermediate_dim): # in MLP: intermediate_dim= 4 * hidden_dim
        super(LLaMAMLP, self).__init__()
        self.linear_1 = nn.Linear(hidden_dim, intermediate_dim)
        self.linear_2 = nn.Linear(hidden_dim, intermediate_dim) # Original: intermediate -> hidden.
        self.activation_fn = nn.SiLU()
        self.out_proj = nn.Linear(intermediate_dim, hidden_dim) # Original: dropout


    def forward(self, hidden_states):
        x_fc_1 = self.linear_1(hidden_states)
        x_fc_2 = self.linear_2(hidden_states)
        x = self.activation_fn(x_fc_1) * x_fc_2
        return self.out_proj(x)

In [12]:
d_model = config.hidden_size
intermediate_dim = config.intermediate_size
mlp = LLaMAMLP(d_model, intermediate_dim).to('cuda')
mlp(input_data).shape

torch.Size([5, 512, 1024])

In [13]:
count_parameters(mlp)

'BabyLlama size: 6.30M parameters'

In [14]:
class Block(nn.Module):
    def __init__(self, config: BabyLlamaConfig):
        super(Block, self).__init__()
        self.hidden_dim = hidden_dim = config.hidden_size
        self.intermediate_dim = intermediate_dim = config.intermediate_size

        self.input_layernorm = LlamaRMSNorm(hidden_dim, eps=config.rms_norm_eps)
        self.attn = BabyLlamaAttention(config)
        self.post_attention_layernorm = LlamaRMSNorm(hidden_dim, eps=config.rms_norm_eps)
        self.mlp = LLaMAMLP(hidden_dim, intermediate_dim)

    def forward(
        self,
        hidden_states,
        mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
    ):
        # 1. Normalizacja wejścia + atencja
        normed_hidden = self.input_layernorm(hidden_states)
        attn_out = self.attn(normed_hidden, mask, position_ids)
        h = hidden_states + attn_out  # skip connection

        # 2. Normalizacja + MLP
        normed_h = self.post_attention_layernorm(h)
        mlp_out = self.mlp(normed_h)
        out = h + mlp_out  # kolejne skip connection

        return out


In [15]:
class LlamaRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True) # (1/n) * Σ x_i^2
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)

class Block(nn.Module):
    def __init__(self, config: BabyLlamaConfig):
        super(Block, self).__init__()
        self.hidden_dim = hidden_dim = config.hidden_size
        self.intermediate_dim = intermediate_dim = config.intermediate_size

        self.attn = BabyLlamaAttention(config)

        self.mlp = LLaMAMLP(hidden_dim, intermediate_dim)
        self.input_layernorm = LlamaRMSNorm(hidden_dim, eps=config.rms_norm_eps)
        self.post_attention_layernorm = LlamaRMSNorm(hidden_dim, eps=config.rms_norm_eps)


    def forward(
        self,
        hidden_states,
        mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
    ):
        r = self.attn(self.input_layernorm(hidden_states), mask,position_ids,)
        h = hidden_states + r
        r = self.mlp(self.post_attention_layernorm(h))
        out = h + r
        return out


In [16]:
block = Block(config).to('cuda')
block(input_data).shape

torch.Size([5, 512, 1024])

In [17]:
count_parameters(block)

'BabyLlama size: 8.66M parameters'

In [18]:
class BabyLlamaModel(nn.Module):
    def __init__(self, config):
        super(BabyLlamaModel, self).__init__()
        self.config = config
        self.hidden_dim = hidden_dim = config.hidden_size
        self.vocab_size = vocab_size = config.vocab_size
        assert self.vocab_size > 0
        self.num_hidden_layers = num_hidden_layers = config.num_hidden_layers

        self.embed_tokens = nn.Embedding(vocab_size, hidden_dim)
        self.blocks = nn.ModuleList(
            [Block(config) for _ in range(num_hidden_layers)]
        )
        self.norm = LlamaRMSNorm(hidden_dim, eps=config.rms_norm_eps)

    def forward(
        self,
        hidden_states: torch.Tensor,
        mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
    ):

        x = self.embed_tokens(hidden_states)

        seq_len = hidden_states.size(1)
        if position_ids is None:
            position_ids = torch.arange(seq_len, dtype=torch.long, device=self.config.device).unsqueeze(0)


        for b in self.blocks:
            x = b(x, mask, position_ids)

        return self.norm(x)


class BabyLlamaPreTrainedModel(PreTrainedModel):
    config_class = BabyLlamaConfig
    base_model_prefix = "model"
    supports_gradient_checkpointing = True
    _skip_keys_device_placement = "past_key_values"

    def _init_weights(self, module):
        std = self.config.initializer_range
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()


class BabyLlamaForCausalLM(BabyLlamaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.model = BabyLlamaModel(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, self.vocab_size, bias=config.lm_head_bias)
        self.post_init()

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        labels: Optional[torch.LongTensor] = None,
    ):

        outputs = self.model(
            hidden_states=input_ids,
            mask=attention_mask,
            position_ids=position_ids,
        )
        logits = self.lm_head(outputs)
        logits = logits.float()

        loss = None
        if labels is not None:
            # shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous() # all elements expect the last one
            shift_labels = labels[..., 1:].contiguous() # all elements except the first
            # Flatten the tokens
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Ensure tensors are on the same device
            shift_labels = shift_labels.to(shift_logits.device)
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits, shift_labels)


        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
        )


In [19]:
device='cuda' if torch.cuda.is_available() else 'cpu'
config = BabyLlamaConfig(device=device)
llm = BabyLlamaForCausalLM(config).to(device)

input_ids = torch.randint(1, config.vocab_size, (batch_size, sequence_length), device=config.device)

In [20]:
outputs = llm(input_ids, labels=input_ids)

In [21]:
visualize_model(llm.model, outputs['logits'])

In [22]:
outputs

CausalLMOutputWithPast(loss=tensor(10.6078, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[ 4.7978e-01,  4.6098e-01, -3.9299e-01,  ..., -8.0760e-01,
          -2.7878e-01, -6.7098e-02],
         [-1.2264e-01,  2.2488e-01, -4.2673e-01,  ..., -1.3213e+00,
           4.4640e-02, -6.6407e-03],
         [-8.9412e-01,  1.8639e-01, -4.7329e-01,  ..., -7.3474e-01,
           2.5651e-01, -3.9300e-02],
         ...,
         [ 3.2651e-01,  5.7790e-01, -4.0266e-01,  ..., -4.8346e-01,
          -8.4305e-01,  4.0051e-01],
         [ 2.9480e-01,  6.4835e-01, -1.6752e-01,  ..., -2.4249e-01,
          -4.6232e-01, -2.3796e-02],
         [ 2.3581e-01, -8.8158e-02, -2.9405e-02,  ..., -3.5446e-01,
          -7.4926e-02,  5.6053e-01]],

        [[-8.1554e-01, -1.3169e-01, -3.8708e-04,  ..., -8.5074e-01,
          -7.5781e-01,  1.7231e-01],
         [-7.7250e-01, -1.4461e-01,  8.6560e-03,  ..., -2.1231e-01,
          -1.0754e+00,  2.3065e-01],
         [-1.1015e+00, -4.3651e-01,  1.8494e-01

In [23]:
llm

BabyLlamaForCausalLM(
  (model): BabyLlamaModel(
    (embed_tokens): Embedding(32000, 1024)
    (blocks): ModuleList(
      (0-5): 6 x Block(
        (attn): BabyLlamaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1024, out_features=128, bias=False)
          (v_proj): Linear(in_features=1024, out_features=128, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): RotaryPositionalEmbeddings()
        )
        (mlp): LLaMAMLP(
          (linear_1): Linear(in_features=1024, out_features=2048, bias=True)
          (linear_2): Linear(in_features=1024, out_features=2048, bias=True)
          (activation_fn): SiLU()
          (out_proj): Linear(in_features=2048, out_features=1024, bias=True)
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear

In [24]:
count_parameters(llm)

'BabyLlama size: 117.48M parameters'

## Dataset

In [25]:
from datasets import load_dataset

torch.manual_seed(64)

train_dataset = load_dataset("huggingface-course/codeparrot-ds-train", split="train[:1%]")
val_dataset = load_dataset("huggingface-course/codeparrot-ds-valid")


In [26]:
train_dataset

Dataset({
    features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
    num_rows: 6067
})

### Tokenizer

In [27]:
from transformers import AutoTokenizer
from huggingface_hub import login

login()

In [28]:
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", add_eos_token = True)
#tokenizer = AutoTokenizer.from_pretrained("gpt2")
#tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")
tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b_v2", legacy=False)
print(tokenizer.vocab_size)  # Powinno dać 32000

32000


In [29]:
tokenizer

LlamaTokenizerFast(name_or_path='openlm-research/open_llama_3b_v2', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [30]:
tokenizer.pad_token = tokenizer.unk_token

In [31]:
text = "Hi I'm Pawel Zmudzki"
tokenizer(text).tokens()

['<s>', '▁Hi', '▁I', "'", 'm', '▁Paw', 'el', '▁Z', 'm', 'ud', 'z', 'ki']

In [32]:
context_length = 10
tokens = tokenizer(
    #train_dataset["train"][1]["content"],
    train_dataset[1]["content"],
    padding=True,
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(tokens['input_ids'])}")
print(f"Input chunk lengths: {(tokens['length'])}")
print(f"Chunk mapping: {tokens['overflow_to_sample_mapping']}")

Input IDs length: 232
Input chunk lengths: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
Chunk mapping: [0, 0, 0, 0, 

In [33]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="pt")

In [34]:
sample = data_collator.torch_call([tokens])

In [35]:
sample

{'input_ids': tensor([[[    1, 11414,    13,  ..., 24842,    13,  8315],
         [    1,  5992,   293,  ...,  2184,  1104,  1048],
         [    1,   268,  8644,  ..., 29531,  4491, 29533],
         ...,
         [    1, 23218,  4322,  ...,    13,   498, 29512],
         [    1,  3309, 29554,  ..., 29520, 29544,  4833],
         [    0,     0,     0,  ...,     1, 29533,    13]]]), 'attention_mask': tensor([[[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]]]), 'length': tensor([[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 10,

In [36]:
sample['input_ids'][0].shape

torch.Size([232, 10])

In [37]:
#outputs = llm(input_ids=sample['input_ids'][0], labels=sample['labels'][0])
outputs = llm(
    input_ids=sample['input_ids'][0].to('cuda'),
    labels=sample['labels'][0].to('cuda')
)

In [38]:
outputs

CausalLMOutputWithPast(loss=tensor(10.6222, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[ 1.4927e-01,  6.2134e-01, -7.8986e-01,  ..., -5.5053e-01,
           5.6152e-02, -6.4053e-01],
         [-1.8564e-01,  3.8641e-01, -1.4469e+00,  ..., -5.1198e-01,
           2.7422e-01,  2.2504e-01],
         [ 1.0556e-02,  2.4785e-01, -1.1464e+00,  ..., -8.1059e-01,
           3.1138e-01,  7.8489e-01],
         ...,
         [-3.7450e-01, -3.3516e-02, -1.4153e+00,  ..., -1.0825e+00,
           1.3368e-01,  8.0685e-01],
         [-2.8954e-01, -1.9320e-02, -1.3432e+00,  ..., -1.2068e+00,
           2.2533e-01,  1.0597e+00],
         [-2.2903e-01,  2.0719e-01, -1.5604e+00,  ..., -1.2028e+00,
          -7.0816e-02,  9.4720e-01]],

        [[ 1.4927e-01,  6.2134e-01, -7.8986e-01,  ..., -5.5053e-01,
           5.6152e-02, -6.4053e-01],
         [-3.0673e-01,  4.2433e-01, -1.0667e+00,  ..., -3.3029e-01,
           6.9872e-01, -5.5264e-01],
         [-5.1567e-01,  4.2312e-01, -1.1435e+00

In [39]:
outputs.logits.shape

torch.Size([232, 10, 32000])

In [40]:
torch.argmax(F.softmax(outputs.logits[0], dim=-1), dim=-1)

tensor([  351, 30918, 30918, 16112, 16112, 16112, 22015,  1595,  1595, 27705],
       device='cuda:0')

In [41]:
tokenizer.batch_decode(torch.argmax(F.softmax(outputs.logits[:, -1, :], dim=-1), dim=-1))

['oner',
 'implement',
 'Worker',
 'familiar',
 'ears',
 'oggle',
 'average',
 'Patrick',
 'limits',
 'gene',
 'Attribute',
 'Royal',
 'oggle',
 'wer',
 'Sha',
 'freight',
 'prolong',
 'blamed',
 'Engine',
 'reluct',
 'hasn',
 'ears',
 '订',
 'shirts',
 '})',
 '户',
 'index',
 'motions',
 'admin',
 'Terms',
 'ouble',
 'provid',
 'ো',
 'inhab',
 'Example',
 '})',
 'PDF',
 'should',
 'aw',
 'ox',
 'fulfill',
 'Bedroom',
 'wer',
 'recipe',
 'barr',
 'Maur',
 'className',
 'advance',
 'cursor',
 'ensuring',
 'Hort',
 'Tor',
 'raisal',
 'enclosed',
 'ulia',
 'wallpaper',
 'exciting',
 'reluct',
 'ries',
 'ul',
 'usk',
 'flies',
 'Tribune',
 'nam',
 '究',
 'reluct',
 '\u200e',
 'opt',
 'babe',
 'prolong',
 'Influ',
 'DAM',
 'ylon',
 'enclosed',
 'arch',
 'irr',
 'Sha',
 'aren',
 'Changing',
 'as',
 'invest',
 'Pass',
 'upwards',
 'rive',
 'hh',
 'imports',
 'ｒ',
 'animation',
 'taining',
 'intervention',
 'Publishing',
 'ref',
 'edition',
 'Sha',
 'eyel',
 'heights',
 'Reb',
 'раз',
 'ini',
 'S

In [42]:
def tokenize(item):
    context_length = config.max_position_embeddings
    outputs = tokenizer(
        item['content'],
        padding=True,
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)

    return {"input_ids": input_batch}

In [43]:
tokenized_train_dataset = train_dataset.map(
    tokenize, batched=True, remove_columns=train_dataset.column_names
)

In [44]:
flush()

## Training

In [45]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "./babyLlama",
    per_device_train_batch_size=16,  # zmniejszone ze względu na pamięć GPU (4GB RAM)
    max_steps=2000,  #20000
    logging_steps=10,
    gradient_accumulation_steps=2,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    fp16=True,
    optim="adamw_torch",
    push_to_hub=False,
)

In [46]:
trainer = Trainer(
    model=llm,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    tokenizer=tokenizer  # poprawnie: to nadal działa mimo ostrzeżenia
)

C:\Users\pz197\AppData\Local\Temp\ipykernel_18320\1103353005.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [47]:
trainer.train()

Step,Training Loss
10,10.551100
20,10.101500
30,9.267900
40,8.502500
50,8.019000
60,7.560400
70,7.116200
80,6.671500
90,6.322800
100,6.029200


TrainOutput(global_step=2000, training_loss=2.932010573387146, metrics={'train_runtime': 77572.8254, 'train_samples_per_second': 0.825, 'train_steps_per_second': 0.026, 'total_flos': 1.6651265796734976e+16, 'train_loss': 2.932010573387146, 'epoch': 1.1415929203539823})

In [48]:
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b_v2")


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

In [49]:
tokenizer.pad_token = tokenizer.eos_token

In [50]:
tokens = tokenizer(
    "import numpy",
    return_tensors='pt'
).to('cuda')
input_ids = tokens['input_ids']

temperature = 1
top_k = None
top_p = None

# Generate the tokens one by one
for _ in range(10):
    # Get the logits from the model
    outputs = llm(input_ids)
    logits = outputs.logits[:, -1, :]

    # Apply temperature scaling
    logits = logits / temperature

    # Apply top-k or top-p sampling if specified
    if top_k is not None:
        logits = logits.topk(top_k, dim=-1)[0]
    elif top_p is not None:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
        sorted_indices_to_remove[:, 0] = 0
        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        logits = logits.masked_fill(indices_to_remove, -float('inf'))

    # Sample the next token from the logits
    next_token_id = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)

    # Update the input with the new token
    input_ids = torch.cat([input_ids, next_token_id], dim=-1)

# Decode the generated text
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)


In [51]:
count_parameters(llm)

'BabyLlama size: 117.48M parameters'

In [52]:
print(generated_text)

import numpy as np
from scipy import stats as


In [53]:
trainer.push_to_hub()

events.out.tfevents.1748711417.Lenovo-Legion.18320.0:   0%|          | 0.00/47.4k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/470M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pawelzm/babyLlama/commit/7fbf74a34506770f8c0d3e266a629af7c3ad6e2c', commit_message='End of training', commit_description='', oid='7fbf74a34506770f8c0d3e266a629af7c3ad6e2c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pawelzm/babyLlama', endpoint='https://huggingface.co', repo_type='model', repo_id='pawelzm/babyLlama'), pr_revision=None, pr_num=None)

## Register Config and Model to HF Auto Class

If your model is very similar to a model inside the library, you can re-use the same configuration as this model.

In [54]:
from transformers import CONFIG_MAPPING, MODEL_MAPPING

# Zarejestruj własny config pod unikalnym typem
CONFIG_MAPPING.register("baby-llama", BabyLlamaConfig)
# Zmapuj config na własny model
MODEL_MAPPING.register(BabyLlamaConfig, BabyLlamaModel)
